# PubliPlots ExamplesComprehensive guide to creating publication-ready visualizations with PubliPlots.---## Table of Contents1. [Setup](#1-setup)2. [Bar Plots](#2-bar-plots)3. [Hatch Pattern Modes](#3-hatch-pattern-modes)4. [Scatter Plots](#4-scatter-plots)5. [Bubble Plots](#5-bubble-plots)6. [Box Plots](#6-box-plots)7. [Violin Plots](#7-violin-plots)8. [Swarm Plots](#8-swarm-plots)9. [Venn Diagrams](#9-venn-diagrams)10. [UpSet Plots](#10-upset-plots)11. [Configuration with pp.rcParams](#11-configuration-with-pprcparams)---

## 1. SetupImport libraries and set the publication style globally.

In [ ]:
import publiplots as ppimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltpp.set_notebook_style()%matplotlib inlineprint(f"PubliPlots version: {pp.__version__}")print("Setup complete!")

---## 2. Bar PlotsThe `barplot()` function creates publication-ready bar plots with support for grouping, error bars, and hatch patterns.

### 2.1 Simple Bar PlotBasic bar plot with categorical data.

In [ ]:
simple_data = pd.DataFrame({    'category': ['A', 'B', 'C', 'D', 'E'],    'value': [23, 45, 38, 52, 41]})fig, ax = pp.barplot(    data=simple_data,    x='category',    y='value',    title='Simple Bar Plot',    xlabel='Category',    ylabel='Value',    palette='pastel',)

### 2.2 Bar Plot with Error BarsBar plot showing mean ± standard error with multiple measurements per category.

In [ ]:
np.random.seed(42)error_data = pd.DataFrame({    'treatment': np.repeat(['Control', 'Drug A', 'Drug B', 'Drug C'], 12),    'response': np.concatenate([        np.random.normal(100, 15, 12),        np.random.normal(120, 12, 12),        np.random.normal(135, 18, 12),        np.random.normal(110, 14, 12),    ])})fig, ax = pp.barplot(    data=error_data,    x='treatment',    y='response',    title='Drug Response with Standard Error',    xlabel='Treatment',    ylabel='Response (a.u.)',    errorbar='se',    capsize=0.1,    palette='pastel',)

### 2.3 Bar Plot with Hue (Color Groups)Grouped bar plot using the `hue` parameter to split bars by a grouping variable.

In [ ]:
np.random.seed(123)hue_data = pd.DataFrame({    'time': np.repeat(['Day 1', 'Day 2', 'Day 3', 'Day 4'], 20),    'group': np.tile(np.repeat(['Control', 'Treated'], 10), 4),    'measurement': np.concatenate([        np.random.normal(50, 8, 10), np.random.normal(52, 8, 10),        np.random.normal(52, 9, 10), np.random.normal(65, 10, 10),        np.random.normal(54, 9, 10), np.random.normal(78, 12, 10),        np.random.normal(55, 10, 10), np.random.normal(85, 14, 10),    ])})fig, ax = pp.barplot(    data=hue_data,    x='time',    y='measurement',    hue='group',    title='Time Course: Control vs Treated',    xlabel='Time Point',    ylabel='Measurement',    errorbar='se',    palette={'Control': '#8E8EC1', 'Treated': '#75B375'},)

### 2.4 Bar Plot with Hatch PatternsUsing hatch patterns to differentiate groups.

In [ ]:
np.random.seed(456)hatch_data = pd.DataFrame({    'condition': np.repeat(['Low', 'Medium', 'High'], 15),    'intensity': np.concatenate([        np.random.normal(30, 5, 15),        np.random.normal(60, 8, 15),        np.random.normal(90, 10, 15),    ])})fig, ax = pp.barplot(    data=hatch_data,    x='condition',    y='intensity',    hatch='condition',    title='Intensity by Condition (Hatch Patterns)',    xlabel='Condition',    ylabel='Intensity',    errorbar='se',    color='#5D83C3',    hatch_map={'Low': '', 'Medium': '//', 'High': 'xx'},    alpha=0.0,)

### 2.5 Bar Plot with Hue + Hatch (Double Split)Combining both color grouping and pattern differentiation.

In [ ]:
np.random.seed(789)double_split_data = pd.DataFrame({    "cell_type": np.repeat(["TypeA", "TypeB", "TypeC"], 40),    "treatment": np.tile(np.repeat(["Vehicle", "Drug"], 20), 3),    "time": np.tile(np.repeat(["24h", "48h"], 10), 6),    "viability": np.concatenate([        np.random.normal(95, 5, 10), np.random.normal(93, 5, 10),        np.random.normal(75, 8, 10), np.random.normal(60, 10, 10),        np.random.normal(94, 5, 10), np.random.normal(92, 5, 10),        np.random.normal(80, 8, 10), np.random.normal(70, 9, 10),        np.random.normal(96, 4, 10), np.random.normal(95, 4, 10),        np.random.normal(85, 7, 10), np.random.normal(78, 8, 10),    ])})fig, ax = pp.barplot(    data=double_split_data,    x="cell_type",    y="viability",    hue="treatment",    hatch="time",    title="Cell Viability: Treatment × Time × Cell Type",    xlabel="Cell Type",    ylabel="Viability (%)",    errorbar="se",    palette={"Vehicle": "#8E8EC1", "Drug": "#E67E7E"},    hatch_map={"24h": "", "48h": "///"},    figsize=(8, 5))

### 2.6 Horizontal Bar PlotCreate horizontal bars by swapping x and y axes.

In [ ]:
np.random.seed(111)horizontal_data = pd.DataFrame({    'gene': ['Gene A', 'Gene B', 'Gene C', 'Gene D', 'Gene E', 'Gene F'],    'expression': np.random.uniform(50, 200, 6),    'group': ['Upregulated', 'Upregulated', 'Downregulated',               'Upregulated', 'Downregulated', 'Upregulated']})fig, ax = pp.barplot(    data=horizontal_data,    x='expression',    y='gene',    hue='group',    title='Gene Expression Levels (Horizontal)',    xlabel='Expression Level',    ylabel='Gene',    palette={'Upregulated': '#75B375', 'Downregulated': '#E67E7E'},    errorbar=None,    alpha=0.3,)

---## 3. Hatch Pattern ModesPubliPlots supports multiple hatch pattern density modes.

### 3.1 Comparing Hatch Modes

In [ ]:
np.random.seed(333)hatch_mode_data = pd.DataFrame({    'sample': np.repeat(['S1', 'S2', 'S3'], 10),    'value': np.concatenate([        np.random.normal(50, 5, 10),        np.random.normal(70, 6, 10),        np.random.normal(90, 7, 10),    ])})fig, axes = plt.subplots(2, 2, figsize=(8, 6), sharex=True, sharey=True)kwargs = dict(data=hatch_mode_data, x='sample', y='value', hatch='sample',              xlabel='Sample', ylabel='', color='#5D83C3', errorbar='se')for i, (ax, mode) in enumerate(zip(axes.flat, [1, 2, 3, 4])):    pp.set_hatch_mode(mode)    pp.barplot(**kwargs, title=f'Mode {mode}', ax=ax)pp.set_hatch_mode()plt.tight_layout()

---## 4. Scatter PlotsThe `scatterplot()` function creates scatter plots with support for size and color encoding.

### 4.1 Basic Scatter Plot

In [ ]:
np.random.seed(444)n = 100scatter_data = pd.DataFrame({    'x': np.random.normal(50, 20, n),    'y': np.random.normal(100, 30, n)})fig, ax = pp.scatterplot(    data=scatter_data,    x='x',    y='y',    title='Basic Scatter Plot',    xlabel='X Variable',    ylabel='Y Variable',)ax.margins(x=0.1, y=0.1)

### 4.2 Scatter with Size Encoding

In [ ]:
scatter_data['magnitude'] = np.abs(scatter_data['x'] - 50) + np.abs(scatter_data['y'] - 100)fig, ax = pp.scatterplot(    data=scatter_data,    x='x',    y='y',    size='magnitude',    sizes=(50, 500),    title='Scatter Plot with Size Encoding',    xlabel='X Variable',    ylabel='Y Variable',)ax.margins(x=0.1, y=0.1)

### 4.3 Scatter with Categorical Hue

In [ ]:
scatter_data['group'] = pd.cut(scatter_data['y'], bins=3, labels=['Low', 'Medium', 'High'])fig, ax = pp.scatterplot(    data=scatter_data,    x='x',    y='y',    hue='group',    palette='pastel',    title='Scatter Plot with Categorical Groups',    xlabel='X Variable',    ylabel='Y Variable',    alpha=0.2,)

### 4.4 Scatter with Continuous Hue

In [ ]:
scatter_data['score'] = scatter_data['x'] * 0.5 + scatter_data['y'] * 0.3 + np.random.randn(n) * 10fig, ax = pp.scatterplot(    data=scatter_data,    x='x',    y='y',    hue='score',    size='magnitude',    palette='viridis',    hue_norm=(scatter_data['score'].min(), scatter_data['score'].max()),    title='Scatter Plot with Continuous Color Scale',    xlabel='X Variable',    ylabel='Y Variable',    alpha=0.2,    figsize=(8, 6),)

---## 5. Bubble PlotsScatter-based heatmaps with categorical axes.

### 5.1 Basic Bubble Plot

In [ ]:
np.random.seed(555)conditions = ['Ctrl', 'Trt1', 'Trt2', 'Trt3']cell_types = ['TypeA', 'TypeB', 'TypeC', 'TypeD']heatmap_data = []for condition in conditions:    for cell_type in cell_types:        heatmap_data.append({            'condition': condition,            'cell_type': cell_type,            'pvalue': np.random.uniform(0.5, 5),            'category': np.random.choice(['Up', 'Down', 'Neutral'])        })heatmap_df = pd.DataFrame(heatmap_data)fig, ax = pp.scatterplot(    data=heatmap_df,    x='condition',    y='cell_type',    size='pvalue',    hue='category',    sizes=(100, 600),    palette={'Up': '#75B375', 'Down': '#E67E7E', 'Neutral': '#CCCCCC'},    title='Differential Expression Analysis',    xlabel='Condition',    ylabel='Cell Type',    margins=0.2)

### 5.2 Bubble Plot with Continuous Colors

In [ ]:
heatmap_df['log2fc'] = np.random.uniform(-3, 3, len(heatmap_df))fig, ax = pp.scatterplot(    data=heatmap_df,    x='condition',    y='cell_type',    size='pvalue',    hue='log2fc',    sizes=(50, 500),    palette='RdBu_r',    hue_norm=(-3, 3),    title='Log2 Fold Change Heatmap',    xlabel='Condition',    ylabel='Cell Type',    alpha=0.2,)

---## 6. Box PlotsThe `boxplot()` function creates publication-ready box plots with transparent fill and opaque edges.

### 6.1 Simple Box Plot

In [ ]:
np.random.seed(42)box_data = pd.DataFrame({    'treatment': np.repeat(['Control', 'Drug A', 'Drug B', 'Drug C'], 30),    'response': np.concatenate([        np.random.normal(100, 15, 30),        np.random.normal(120, 12, 30),        np.random.normal(135, 18, 30),        np.random.normal(110, 14, 30),    ])})fig, ax = pp.boxplot(    data=box_data,    x='treatment',    y='response',    title='Drug Response Distribution',    xlabel='Treatment',    ylabel='Response (a.u.)',    palette='pastel',)

### 6.2 Box Plot with Hue

In [ ]:
np.random.seed(123)grouped_box_data = pd.DataFrame({    'time': np.repeat(['Day 1', 'Day 3', 'Day 7'], 40),    'group': np.tile(np.repeat(['Control', 'Treated'], 20), 3),    'measurement': np.concatenate([        np.random.normal(50, 8, 20), np.random.normal(52, 8, 20),        np.random.normal(52, 9, 20), np.random.normal(70, 12, 20),        np.random.normal(55, 10, 20), np.random.normal(85, 14, 20),    ])})fig, ax = pp.boxplot(    data=grouped_box_data,    x='time',    y='measurement',    hue='group',    gap=0.1,    title='Time Course: Control vs Treated',    xlabel='Time Point',    ylabel='Measurement',    palette={'Control': '#8E8EC1', 'Treated': '#75B375'},)

### 6.3 Box Plot with Custom Styling

In [ ]:
fig, ax = pp.boxplot(    data=box_data,    x='treatment',    y='response',    hue='treatment',    title='Drug Response with Color by Treatment',    xlabel='Treatment',    ylabel='Response (a.u.)',    palette=['#75b375', '#e6b375', '#e67e7e', '#8e8ec1'],    showcaps=False,)

---## 7. Violin PlotsThe `violinplot()` function creates publication-ready violin plots showing full distribution shapes.

### 7.1 Simple Violin Plot

In [ ]:
np.random.seed(42)violin_data = pd.DataFrame({    'treatment': np.repeat(['Control', 'Drug A', 'Drug B', 'Drug C'], 30),    'response': np.concatenate([        np.random.normal(100, 15, 30),        np.random.normal(120, 12, 30),        np.random.normal(135, 18, 30),        np.random.normal(110, 14, 30),    ])})fig, ax = pp.violinplot(    data=violin_data,    x='treatment',    y='response',    title='Drug Response Distribution',    xlabel='Treatment',    ylabel='Response (a.u.)',    palette='pastel',)

### 7.2 Violin Plot with Hue

In [ ]:
np.random.seed(123)grouped_violin_data = pd.DataFrame({    'time': np.repeat(['Day 1', 'Day 3', 'Day 7'], 40),    'group': np.tile(np.repeat(['Control', 'Treated'], 20), 3),    'measurement': np.concatenate([        np.random.normal(50, 8, 20), np.random.normal(52, 8, 20),        np.random.normal(52, 9, 20), np.random.normal(70, 12, 20),        np.random.normal(55, 10, 20), np.random.normal(85, 14, 20),    ])})fig, ax = pp.violinplot(    data=grouped_violin_data,    x='time',    y='measurement',    hue='group',    gap=0.1,    title='Time Course: Control vs Treated',    xlabel='Time Point',    ylabel='Measurement',    palette={'Control': '#8E8EC1', 'Treated': '#75B375'},)

### 7.3 Split Violin Plot

In [ ]:
fig, ax = pp.violinplot(    data=grouped_violin_data,    x='time',    y='measurement',    hue='group',    split=True,    inner='quart',    gap=0.1,    title='Split Violin Plot',    xlabel='Time Point',    ylabel='Measurement',    palette={'Control': '#8E8EC1', 'Treated': '#75B375'},)

### 7.4 Violin Plot Without Fill

In [ ]:
fig, ax = pp.violinplot(    data=grouped_violin_data,    x='time',    y='measurement',    hue='group',    fill=False,    split=True,    inner='quart',    gap=0.1,    title='Violin Plot Without Fill',    xlabel='Time Point',    ylabel='Measurement',    palette={'Control': '#8E8EC1', 'Treated': '#75B375'},)

### 7.5 Violin Plot with Different Inner Representations

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))inner_types = ['box', 'quart', 'stick', 'point']for ax, inner in zip(axes.flat, inner_types):    pp.violinplot(        data=violin_data,        x='treatment',        y='response',        inner=inner,        ax=ax,        title=f'Inner: {inner}',        xlabel='Treatment',        ylabel='Response',        palette='pastel',    )plt.tight_layout()

---## 8. Swarm PlotsThe `swarmplot()` function creates swarm plots showing individual data points without overlap.

### 8.1 Simple Swarm Plot

In [ ]:
np.random.seed(42)swarm_data = pd.DataFrame({    'category': np.repeat(['A', 'B', 'C', 'D'], 25),    'value': np.concatenate([        np.random.normal(50, 10, 25),        np.random.normal(65, 12, 25),        np.random.normal(55, 8, 25),        np.random.normal(70, 15, 25),    ])})fig, ax = pp.swarmplot(    data=swarm_data,    x='category',    y='value',    title='Distribution by Category',    xlabel='Category',    ylabel='Value',    palette='pastel',)

### 8.2 Swarm Plot with Hue

In [ ]:
np.random.seed(456)grouped_swarm_data = pd.DataFrame({    'condition': np.repeat(['Low', 'Medium', 'High'], 40),    'group': np.tile(np.repeat(['Control', 'Treated'], 20), 3),    'response': np.concatenate([        np.random.normal(30, 5, 20), np.random.normal(35, 6, 20),        np.random.normal(50, 7, 20), np.random.normal(60, 8, 20),        np.random.normal(70, 8, 20), np.random.normal(85, 10, 20),    ])})fig, ax = pp.swarmplot(    data=grouped_swarm_data,    x='condition',    y='response',    hue='group',    dodge=True,    title='Response by Condition and Group',    xlabel='Condition',    ylabel='Response',    palette={'Control': '#8E8EC1', 'Treated': '#75B375'},)

### 8.3 Combined Box and Swarm PlotOverlay swarm on box plot for publication-quality figures.

In [ ]:
np.random.seed(789)combined_data = pd.DataFrame({    'treatment': np.repeat(['Control', 'Drug A', 'Drug B'], 25),    'response': np.concatenate([        np.random.normal(100, 12, 25),        np.random.normal(125, 15, 25),        np.random.normal(140, 18, 25),    ])})fig, ax = plt.subplots(figsize=(6, 5))pp.boxplot(    data=combined_data,    x='treatment',    y='response',    hue='treatment',    ax=ax,    palette=['#75b375', '#e6b375', '#e67e7e'],    showcaps=False,    showfliers=False,)pp.swarmplot(    data=combined_data,    x='treatment',    y='response',    hue='treatment',    ax=ax,    palette=['#75b375', '#e6b375', '#e67e7e'],    alpha=1,    legend=False,)ax.set_title('Combined Box and Swarm Plot')ax.set_xlabel('Treatment')ax.set_ylabel('Response (a.u.)')plt.tight_layout()

---## 9. Venn DiagramsThe `venn()` function creates 2 to 5-way Venn diagrams.

### 9.1 2-Way Venn Diagram

In [ ]:
set1 = set(range(1, 51))set2 = set(range(30, 81))fig, ax = pp.venn(    sets=[set1, set2],    labels=['Set A', 'Set B'],    colors=pp.color_palette('pastel', n_colors=2),)

### 9.2 3-Way Venn Diagram

In [ ]:
setA = set(range(1, 61))setB = set(range(40, 101))setC = set(range(51, 131))fig, ax = pp.venn(    sets=[setA, setB, setC],    labels=['Set A', 'Set B', 'Set C'],    colors=pp.color_palette('pastel', n_colors=3),)

### 9.3 4-Way Venn Diagram

In [ ]:
np.random.seed(888)set1 = set(np.random.randint(1, 120, 70))set2 = set(np.random.randint(30, 150, 75))set3 = set(np.random.randint(60, 180, 70))set4 = set(np.random.randint(1, 100, 65))fig, ax = pp.venn(    sets=[set1, set2, set3, set4],    labels=['Dataset A', 'Dataset B', 'Dataset C', 'Dataset D'],    colors=pp.color_palette('pastel', n_colors=4),    figsize=(6, 6),)

### 9.4 5-Way Venn Diagram

In [ ]:
np.random.seed(999)set1 = set(np.random.randint(1, 140, 80))set2 = set(np.random.randint(40, 180, 85))set3 = set(np.random.randint(80, 200, 75))set4 = set(np.random.randint(20, 160, 70))set5 = set(np.random.randint(60, 180, 80))fig, ax = pp.venn(    sets=[set1, set2, set3, set4, set5],    labels=['Group A', 'Group B', 'Group C', 'Group D', 'Group E'],    colors=pp.color_palette('pastel', n_colors=5),    figsize=(6, 6),)

---## 10. UpSet PlotsUpSet plots excel at showing intersections between many sets.

### 10.1 Basic UpSet Plot

In [ ]:
np.random.seed(100)upset_sets = {    'Gene Set A': set(np.random.randint(1, 100, 50)),    'Gene Set B': set(np.random.randint(30, 130, 55)),    'Gene Set C': set(np.random.randint(60, 140, 45)),    'Gene Set D': set(np.random.randint(20, 110, 48))}fig, axes = pp.upsetplot(    data=upset_sets,    title='Gene Set Intersection Analysis',    show_counts=15,)

### 10.2 UpSet Plot with Custom Colors

In [ ]:
fig, axes = pp.upsetplot(    data=upset_sets,    sort_by='size',    title='Customized UpSet Plot',    color='#E67E7E',    alpha=0.3,    bar_linewidth=1.5,    show_counts=12,)

### 10.3 UpSet Plot with Many Sets

In [ ]:
np.random.seed(200)many_sets = {    'Pathway A': set(np.random.randint(1, 120, 60)),    'Pathway B': set(np.random.randint(40, 160, 65)),    'Pathway C': set(np.random.randint(80, 180, 55)),    'Pathway D': set(np.random.randint(20, 140, 58)),    'Pathway E': set(np.random.randint(60, 160, 62)),    'Pathway F': set(np.random.randint(30, 150, 60))}fig, axes = pp.upsetplot(    data=many_sets,    sort_by='size',    title='Pathway Overlap Analysis (6 Pathways)',    color='#75B375',    show_counts=20,)

### 10.4 UpSet Plot from DataFrame

In [ ]:
np.random.seed(300)n_elements = 100upset_df = pd.DataFrame({    'Cluster_1': np.random.choice([0, 1], n_elements, p=[0.6, 0.4]),    'Cluster_2': np.random.choice([0, 1], n_elements, p=[0.5, 0.5]),    'Cluster_3': np.random.choice([0, 1], n_elements, p=[0.7, 0.3]),    'Marker_Set': np.random.choice([0, 1], n_elements, p=[0.65, 0.35]),})fig, axes = pp.upsetplot(    data=upset_df,    sort_by='size',    title='Cluster and Marker Overlaps',    color='#E6B375',    alpha=0.4,    show_counts=15,)

---## 11. Configuration with pp.rcParamsPubliPlots integrates with matplotlib's configuration system through `pp.rcParams`.

### 11.1 Understanding rcParams

In [ ]:
# Access PubliPlots custom parametersprint("PubliPlots Custom Parameters:")print(f"  Default color: {pp.rcParams['color']}")print(f"  Default alpha: {pp.rcParams['alpha']}")print(f"  Default capsize: {pp.rcParams['capsize']}")print(f"  Hatch mode: {pp.rcParams['hatch_mode']}")# Access matplotlib parametersprint("\nMatplotlib Parameters (via pp.rcParams):")print(f"  Figure size: {pp.rcParams['figure.figsize']}")print(f"  Line width: {pp.rcParams['lines.linewidth']}")print(f"  Font size: {pp.rcParams['font.size']}")

### 11.2 Customizing Parameters

In [ ]:
# Customize parameterspp.rcParams['color'] = '#E67E7E'pp.rcParams['alpha'] = 0.3pp.rcParams['figure.figsize'] = (8, 6)pp.rcParams['lines.linewidth'] = 2.0print("Customized Parameters:")print(f"  Color: {pp.rcParams['color']}")print(f"  Alpha: {pp.rcParams['alpha']}")print(f"  Figure size: {pp.rcParams['figure.figsize']}")# Reset to notebook stylepp.set_notebook_style()

---## SummaryThis notebook demonstrated:### Bar Plots- Simple, error bars, hue groups, hatch patterns, hue+hatch, horizontal### Scatter & Bubble Plots- Size encoding, categorical/continuous hue, categorical axes### Box Plots- Distribution statistics, grouped with hue, custom styling### Violin Plots- Full distributions, split violins, different inner representations### Swarm Plots- Individual data points, combined with box plots### Venn & UpSet Plots- 2-5 way Venn diagrams, complex set intersections### Configuration- Unified rcParams interface for customizationFor more information, visit the [PubliPlots GitHub repository](https://github.com/jorgebotas/publiplots).